In [283]:
import pandas as pd
import re
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras import layers
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
np.random.seed(13)

In [6]:
len_elmo = 1024
len_bert = 768
len_w2v = 300
len_glove = 200

#### Lectura BERT

In [7]:
bert_train = np.loadtxt("train_embedding_file_bert.txt").reshape(len(train.index),len_bert)
bert_train.shape

(7613, 768)

In [8]:
bert_test = np.loadtxt("test_embedding_file_bert.txt").reshape(len(test.index),len_bert)
bert_test.shape

(3263, 768)

#### Lectura ELMo

In [9]:
elmo_train = np.loadtxt("train_embedding_file_elmo.txt").reshape(len(train.index),len_elmo)
elmo_train.shape

(7613, 1024)

In [10]:
elmo_test = np.loadtxt("test_embedding_file_elmo.txt").reshape(len(test.index),len_elmo)
elmo_test.shape

(3263, 1024)

#### Lectura W2V

In [11]:
w2v_train = np.loadtxt("train_embedding_file_w2v(norm).txt").reshape(len(train.index),len_w2v)
w2v_train.shape

(7613, 300)

In [12]:
w2v_test = np.loadtxt("test_embedding_file_w2v(norm).txt").reshape(len(test.index),len_w2v)
w2v_test.shape

(3263, 300)

#### Lectura Glove

In [13]:
glove_train = np.loadtxt("train_embedding_file_glove.txt").reshape(len(train.index),len_glove)
glove_train.shape

(7613, 200)

In [14]:
glove_test = np.loadtxt("test_embedding_file_glove.txt").reshape(len(test.index),len_glove)
glove_test.shape

(3263, 200)

# BERT

In [429]:
train = pd.read_csv('csv/train.csv')
# ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
# train.loc[train['id'].isin(ids_with_target_error),'target'] = 0
test = pd.read_csv('csv/test.csv')

In [430]:
X, y = bert_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [431]:
model = Sequential()
# model.add(layers.Dense(round(len_bert*.75), input_dim=len_bert, activation='relu'))
# model.add(Dropout(0.25))
# model.add(layers.Dense(round(len_bert/2), input_dim=len_bert, activation='relu'))
# model.add(Dropout(0.25))
model.add(layers.Dense(1, input_dim=len_bert, activation='sigmoid'))

In [432]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_120 (Dense)            (None, 1)                 769       
Total params: 769
Trainable params: 769
Non-trainable params: 0
_________________________________________________________________


In [433]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
686/686 [==============================] - 1s 2ms/step - loss: 0.5098 - accuracy: 0.7656 - val_loss: 0.4553 - val_accuracy: 0.7927
Epoch 2/10
686/686 [==============================] - 1s 1ms/step - loss: 0.4563 - accuracy: 0.7992 - val_loss: 0.4390 - val_accuracy: 0.8176
Epoch 3/10
686/686 [==============================] - 1s 1ms/step - loss: 0.4425 - accuracy: 0.8054 - val_loss: 0.4350 - val_accuracy: 0.8228
Epoch 4/10
686/686 [==============================] - 1s 1ms/step - loss: 0.4354 - accuracy: 0.8083 - val_loss: 0.4322 - val_accuracy: 0.8136
Epoch 5/10
686/686 [==============================] - 1s 1ms/step - loss: 0.4299 - accuracy: 0.8154 - val_loss: 0.4453 - val_accuracy: 0.8189
Epoch 6/10
686/686 [==============================] - 1s 1ms/step - loss: 0.4267 - accuracy: 0.8120 - val_loss: 0.4279 - val_accuracy: 0.8150
Epoch 7/10
686/686 [==============================] - 1s 1ms/step - loss: 0.4238 - accuracy: 0.8156 - val_loss: 0.4288 - val_accuracy: 0.8215
Epoch 

In [434]:
test_pred = model.predict(bert_test)
test_pred_int = test_pred.round().astype('int')

In [435]:
test['bert_net'] = test_pred
test = test['bert_net'].to_frame()
test.head()

,bert_net
0,0.942535
1,0.976414
2,0.677062
3,0.990948
4,0.947406


In [436]:
test.to_csv('csv/test_red_bert.csv')

In [437]:
train_pred = model.predict(bert_train)
train_pred_int = train_pred.round().astype('int')

In [438]:
train['bert_net'] = train_pred
train = train['bert_net'].to_frame()
train.to_csv('csv/train_red_bert.csv')

# ELMo

In [820]:
train = pd.read_csv('csv/train.csv')
# ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
# train.loc[train['id'].isin(ids_with_target_error),'target'] = 0
test = pd.read_csv('csv/test.csv')
X, y = elmo_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [821]:
model = Sequential()
model.add(layers.Dense(round(len_elmo/2), input_dim=len_elmo, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

In [822]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_82"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_215 (Dense)            (None, 512)               524800    
_________________________________________________________________
dropout_134 (Dropout)        (None, 512)               0         
_________________________________________________________________
dense_216 (Dense)            (None, 1)                 513       
Total params: 525,313
Trainable params: 525,313
Non-trainable params: 0
_________________________________________________________________


In [823]:
history = model.fit(X_train, y_train,
                    epochs=5,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/5
686/686 [==============================] - 4s 5ms/step - loss: 0.4847 - accuracy: 0.7765 - val_loss: 0.4403 - val_accuracy: 0.7992
Epoch 2/5
686/686 [==============================] - 4s 6ms/step - loss: 0.4380 - accuracy: 0.8038 - val_loss: 0.4450 - val_accuracy: 0.8045
Epoch 3/5
686/686 [==============================] - 3s 5ms/step - loss: 0.4179 - accuracy: 0.8120 - val_loss: 0.4278 - val_accuracy: 0.8215
Epoch 4/5
686/686 [==============================] - 3s 5ms/step - loss: 0.3987 - accuracy: 0.8199 - val_loss: 0.4312 - val_accuracy: 0.8018
Epoch 5/5
686/686 [==============================] - 4s 5ms/step - loss: 0.3846 - accuracy: 0.8297 - val_loss: 0.4431 - val_accuracy: 0.8136


In [824]:
test_pred = model.predict(elmo_test)
test_pred_int = test_pred.round().astype('int')

In [825]:
test['elmo_net'] = test_pred
test = test['elmo_net'].to_frame()
test.head()

,elmo_net
0,0.534085
1,0.828935
2,0.471799
3,0.886043
4,0.887910


In [826]:
test.to_csv('csv/test_red_elmo.csv')

In [827]:
train_pred = model.predict(elmo_train)
train_pred_int = train_pred.round().astype('int')

In [828]:
train['elmo_net'] = train_pred
train = train['elmo_net'].to_frame()
train.to_csv('csv/train_red_elmo.csv')

# W2V

In [571]:
train = pd.read_csv('csv/train.csv')
# ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
# train.loc[train['id'].isin(ids_with_target_error),'target'] = 0
test = pd.read_csv('csv/test.csv')
X, y = w2v_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [572]:
model = Sequential()
model.add(layers.Dense(round(len_w2v/2), input_dim=len_w2v, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(round(len_w2v/4), input_dim=len_w2v, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

In [573]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_57"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_156 (Dense)            (None, 150)               45150     
_________________________________________________________________
dropout_100 (Dropout)        (None, 150)               0         
_________________________________________________________________
dense_157 (Dense)            (None, 75)                11325     
_________________________________________________________________
dropout_101 (Dropout)        (None, 75)                0         
_________________________________________________________________
dense_158 (Dense)            (None, 1)                 76        
Total params: 56,551
Trainable params: 56,551
Non-trainable params: 0
_________________________________________________________________


In [574]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
686/686 [==============================] - 2s 2ms/step - loss: 0.4863 - accuracy: 0.7811 - val_loss: 0.4351 - val_accuracy: 0.8176
Epoch 2/10
686/686 [==============================] - 1s 2ms/step - loss: 0.4339 - accuracy: 0.8092 - val_loss: 0.4361 - val_accuracy: 0.8136
Epoch 3/10
686/686 [==============================] - 1s 2ms/step - loss: 0.4014 - accuracy: 0.8234 - val_loss: 0.4255 - val_accuracy: 0.8110
Epoch 4/10
686/686 [==============================] - 1s 2ms/step - loss: 0.3774 - accuracy: 0.8373 - val_loss: 0.4235 - val_accuracy: 0.8163
Epoch 5/10
686/686 [==============================] - 1s 2ms/step - loss: 0.3496 - accuracy: 0.8540 - val_loss: 0.4396 - val_accuracy: 0.8136
Epoch 6/10
686/686 [==============================] - 1s 2ms/step - loss: 0.3242 - accuracy: 0.8641 - val_loss: 0.4552 - val_accuracy: 0.8202
Epoch 7/10
686/686 [==============================] - 1s 2ms/step - loss: 0.2990 - accuracy: 0.8755 - val_loss: 0.4755 - val_accuracy: 0.8202
Epoch 

In [575]:
test_pred = model.predict(w2v_test)
test_pred_int = test_pred.round().astype('int')

In [576]:
test['w2v_net'] = test_pred
test = test['w2v_net'].to_frame()
test.head()

,w2v_net
0,0.921463
1,0.970476
2,0.975260
3,0.999825
4,0.999996


In [577]:
test.to_csv('csv/test_red_w2v.csv')

In [578]:
train_pred = model.predict(w2v_train)
train_pred_int = train_pred.round().astype('int')

In [579]:
train['w2v_net'] = train_pred
train = train['w2v_net'].to_frame()
train.to_csv('csv/train_red_w2v.csv')

# GLOve

In [730]:
train = pd.read_csv('csv/train.csv')
# ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
# train.loc[train['id'].isin(ids_with_target_error),'target'] = 0
test = pd.read_csv('csv/test.csv')
X, y = glove_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [731]:
model = Sequential()
model.add(layers.Dense(round(len_glove/2), input_dim=len_glove, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

In [732]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_73"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_195 (Dense)            (None, 100)               20100     
_________________________________________________________________
dropout_123 (Dropout)        (None, 100)               0         
_________________________________________________________________
dense_196 (Dense)            (None, 1)                 101       
Total params: 20,201
Trainable params: 20,201
Non-trainable params: 0
_________________________________________________________________


In [733]:
history = model.fit(X_train, y_train,
                    epochs=5,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/5
686/686 [==============================] - 1s 2ms/step - loss: 0.5759 - accuracy: 0.7561 - val_loss: 0.4378 - val_accuracy: 0.7992
Epoch 2/5
686/686 [==============================] - 1s 2ms/step - loss: 0.4389 - accuracy: 0.8053 - val_loss: 0.4170 - val_accuracy: 0.8189
Epoch 3/5
686/686 [==============================] - 1s 2ms/step - loss: 0.4051 - accuracy: 0.8244 - val_loss: 0.4099 - val_accuracy: 0.8110
Epoch 4/5
686/686 [==============================] - 1s 2ms/step - loss: 0.3790 - accuracy: 0.8356 - val_loss: 0.4022 - val_accuracy: 0.8255
Epoch 5/5
686/686 [==============================] - 1s 2ms/step - loss: 0.3556 - accuracy: 0.8438 - val_loss: 0.4084 - val_accuracy: 0.8281


In [734]:
test_pred = model.predict(glove_test)
test_pred_int = test_pred.round().astype('int')

In [735]:
test['glove_net'] = test_pred
test = test['glove_net'].to_frame()
test.head()

,glove_net
0,0.743638
1,0.864992
2,0.609823
3,0.909378
4,0.994435


In [736]:
test.to_csv('csv/test_red_glove.csv')

In [737]:
train_pred = model.predict(glove_train)
train_pred_int = train_pred.round().astype('int')

In [738]:
train['glove_net'] = train_pred
train = train['glove_net'].to_frame()
train.to_csv('csv/train_red_glove.csv')